[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hate-alert/Tutorial-ICWSM-2021/blob/main/Demos/Rationale_predictor_demo.ipynb)


# **Rationale and Label predictor for Abusive speech**
> Here, we present a tool which can predict both rationale and labels given a dataset having unknown label. 
This tool is provided by [Hate-alert](https://github.com/hate-alert)

![hate speech](https://www.media-diversity.org/wp-content/uploads/2021/03/shutterstock_1523413514-780x520.jpg)



#**Install necessary modules**
####this cell will install transformers and other necessary packages required for running the code


In [1]:
%%capture
!pip install transformers
!pip install torch
!pip install ekphrasis
!git clone https://github.com/hate-alert/Tutorial-ICWSM-2021.git


In [2]:
cd Tutorial-ICWSM-2021

/content/Tutorial-ICWSM-2021


#### **Import necessary modules**


In [3]:
%%capture
import transformers
from transformers import AutoTokenizer
from transformers import BertForTokenClassification, BertForSequenceClassification,BertPreTrainedModel, BertModel
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import re
import random
import torch.nn.functional as F
import numpy as np
from Code.utils import *
from Code.model import *
from Code.predictions import *

### **Set GPU** : 
> This will select the device based on your current configuration. Select Runtime --> change runtimetype and select GPU as hardware accelerator to use GPU. 

In [4]:
if torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cpu")

# **Model and its origin:-**

*  **The model here was trained using the [hatexplain dataset](https://huggingface.co/datasets/hatexplain)** 

This is a BERT-BASE-UNCASED model trained with label predictor and rationale predictor head

> **Labels**  -- abusive (hateful/offensive) and normal labels 
> **Rationales** -- Highlighted the words which can justify the label selected by the annotators (only for abusive labels)


#### **If used cite this**  
```
@article{mathew2020hatexplain,
  title={HateXplain: A Benchmark Dataset for Explainable Hate Speech Detection},
  author={Mathew, Binny and Saha, Punyajoy and Yimam, Seid Muhie and Biemann, Chris and Goyal, Pawan and Mukherjee, Animesh},
  journal={arXiv preprint arXiv:2012.10289},
  year={2020}
}
```

In [34]:
model = modelPredRationale(model_path='Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two',device=device)

In [82]:
#@title **How do you want to enter text ?**
# @markdown You can either directly enter the text (text input) or uppload from a csv (file)
input_type = "text input" #@param ["file", "text input"]

In [83]:
import io
import pandas as pd

if input_type == "text input":
    text_input = input("Write the post: ")
    dataset=[text_input]
else:
  print("Please upload the csv file you want to get predictions")
  print("Please make sure the column name of the csv should be Index, Sentences")
  from google.colab import files
  uploaded = files.upload()
  dataset = pd.read_csv(io.BytesIO(uploaded[list(uploaded)[0]])).reset_index()

Write the post: you deserve death if you voted yes cruz that dumbass spic was destined to lose


In [84]:
def getDatasetPrediction(dataset,config):
    labels,attention,sents=model.return_rationales(dataset['Sentences'])
    predictions = {}
    for index, row in dataset.iterrows():
        dict1={}
        dict1['Sentence']=row['Sentences']
        dict_labels={}
        for ele in config:
            dict_labels[config[ele]]=round(labels[index][ele],3)
        dict1["Labels"]=dict_labels

        dict1["Tokens"] = sents[index]
        if (np.argmax(labels[index])==0):
          dict1["Rationale"]=list(np.zeros(len(sents[index])))
        else:
          dict1["Rationale"]=attention[index][0:len(dict1["Tokens"])]
        predictions[row['Index']] = dict1
    return predictions

def getRandomTextFromPred(pred = None):
    return random.choice(list(pred.items()))

In [85]:
if input_type == "text input":
    labels,attention,sents=model.return_rationales(dataset)
    
else:
    config=model.config.id2label
    predictions= getDatasetPrediction(dataset,config)

Running eval on test data...


In [86]:
def show_attention_rationale(tokenized_text,attention_vector):
    char_vals = [CharVal(c, v) for c, v in zip(tokenized_text, attention_vector)]
    char_df = pd.DataFrame(char_vals).transpose()
    char_df = char_df.style.applymap(color_charvals_rationale)
    return char_df

In [87]:
### Show a random text with rationales, No rationales will mean the text is not predicted as abusive by the model.
if input_type != "text input":
    pred=getRandomTextFromPred(predictions)
    print(pred)
    char_df=show_attention_rationale(pred[1]['Tokens'],pred[1]['Rationale'])
else:
    pred= {'Normal':labels[0][0], 'Abusive':labels[0][1]}
    if (np.argmax(labels[0])==0):
        attention=[list(np.zeros(len(sents[0])))]
    else:
        pass    
    char_df=show_attention_rationale(sents[0], attention[0])

In [88]:
print("Prediction probablities:", pred)
char_df

Prediction probablities: {'Normal': 0.026733398, 'Abusive': 0.9732666}


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,[CLS],you,deserve,death,if,you,voted,yes,cruz,that,dumb,##ass,sp,##ic,was,destined,to,lose,[SEP]


### **Word of caution**

> Model used here have any trained using a particular dataset and they may carry some bias or errors, they should be only used as a complementary labels in case of any analysis.

#**Download the file generated**
#### Run this cell and select the destination folder.


In [ ]:
from google.colab import files
import json
if input_type != "text input":
  with open('predictions.json', 'w') as f:
      json_string = json.dumps(predictions, cls=NumpyEncoder, sort_keys=True, indent=4)
      f.write(json_string)
  files.download('predictions.json')
else:
  print("No file input given")

No file input given
